# 04 - Preventing overfitting

{TO-DO: Overview of notebook}

## Overfitting/Underfitting and Bias/Variance

- !! Explain what is overfitting and underfitting
    - Show images
- !! Explain Bias and Variance trade-off
    - Show images
    
- We need to test the generalizability of our model, and evaluate its performance with data that has never seen (but comes from the same distribution as the training data)

We can illustrate the problem of overfitting using scikit-learn. Let's create a fake dataset for classification, and fit and score a support vector classifier that uses a polinomial kernel of degree 4:

In [5]:
from sklearn.datasets import make_classification
from sklearn.svm import SVC

# Create fake dataset
X, y = make_classification(
    n_samples=100, n_features=20, n_informative=5, n_redundant=15, random_state=1
)

# Create and fit SVC
svc = SVC(kernel='poly', degree=4, random_state=0).fit(X, y)

# Score model
print(f"SVC mean performance: {svc.score(X, y)}")

SVC mean performance: 0.82


Notice that here and during all the previous examples of this tutorial we have been using the dataset used for training (`X`) to also evaluate the performance of the model. As explained before, this could lead in overestimation of the model perfomance.

Let's instead split our dataset into a training and testing set. We can do so using the method `train_test_split` in scikit-learn (read documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)):

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

print(f"Shape of training set: {X_train.shape}")
print(f"Shape of training labels: {y_train.shape}")
print(f"Shape of testing sett: {X_test.shape}")
print(f"Shape of testing labels: {y_test.shape}")

Shape of training set: (75, 20)
Shape of training labels: (75,)
Shape of testing sett: (25, 20)
Shape of testing labels: (25,)


By default, scikit-learn splits `X` into a training size of 75% and a testing size of 25%.

What happens if we fit our estimator to the training set, and evaluate it both using the training and testing set?

In [10]:
# Fit model to training set
svc = SVC(kernel='poly', degree=5, random_state=0).fit(X_train, y_train)

# Evaluate model with training and testing data
print(f"SVC mean accuracy on training data: {svc.score(X_train, y_train)}")
print(f"SVC mean accuracy on testing data: {svc.score(X_test, y_test)}")

SVC mean accuracy on training data: 0.8
SVC mean accuracy on testing data: 0.52


As you can see, the performance of the model drops significantly when evaluated on the testing set, indicating that our model may have overfitted to the training data.

### Exercise

Suppose we want our testing size to comprise 20% of the original dataset. Modify `train_test_split` to achieve this aim.

#### Answer

----

Fortunately, there exists various methods that help reducing overfitting during training. In this notebook, we will revise two of them: _cross-validation_ and _regularization_.

# Cross-validation

- !! Why use cross-validation
    - reduce overfitting
    - test generalization
- !! Types of cross-validation
- !! Show images of cross-validation
- !! Give as examples how to implement Stratified K Fold

In [127]:
from sklearn.datasets import make_classification

X, y = make_classification(
    n_samples=500, n_features=300, n_informative=100, random_state=0
)

Let's create a stratified cross-validation object using `StratifiedKFold` (read documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold)). We will use 3 splits:

In [120]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)

Let's create a logistic regression model and evaluate the model using the cross-validation object defined above and the `cross_validate` method (read documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

In [121]:
from sklearn.model_selection import cross_validate

clf = LogisticRegression(max_iter=1000)

cv_results = cross_validate(
    clf, X, y, scoring='accuracy', cv=skf
)

Let's inspect the output of the cross-validation procedure:

In [124]:
cv_results

{'fit_time': array([0.01362705, 0.00827885, 0.00480318]),
 'score_time': array([0.00795507, 0.00030303, 0.00038409]),
 'test_score': array([0.91176471, 0.90909091, 0.81818182])}

We can also return the training scores and estimators used in each split:

In [126]:
cv_results = cross_validate(
    clf, X, y, scoring='accuracy', cv=skf,
    return_train_score=True,
    return_estimator=True
)

In [127]:
cv_results

{'fit_time': array([0.00624418, 0.00407386, 0.00546193]),
 'score_time': array([0.00039387, 0.00029588, 0.00043488]),
 'estimator': [LogisticRegression(max_iter=1000),
  LogisticRegression(max_iter=1000),
  LogisticRegression(max_iter=1000)],
 'test_score': array([0.91176471, 0.90909091, 0.81818182]),
 'train_score': array([0.89393939, 0.91044776, 0.94029851])}

In this way we can access the estimator fitted in a specific split of the data, and use it to predict data, for example:

In [132]:
split = 0
estimator_split = cv_results["estimator"][0]
estimator_split.predict(X)

array([0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1])

{TO-DO: show how in the training and testing set we perform the preprocessing steps outside the testing too}

### Exercise

Can you read the documentation of `RepeatedStratifiedKFold` [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedStratifiedKFold.html#sklearn.model_selection.RepeatedStratifiedKFold), understand it and implement it as above?

# Regularization

- !! Another way of preventing overfitting is to regularize our models
- !! Explain what is regularization
    - Regularization penalizes your model during training to avoid overfitting. More specifically, it adds a penalty to the _loss_ function of your model.
- There are two main types of regularization:
    - L1 or _Lasso_: Makes the coefficients sparse, meaning some of them are shrinked to 0.
        - {add formula}
        - {the function looks more simple}
    - L2 or _Ridge_: Makes the coefficients smaller.
        - {add formula}
        - {the function looks smoother}
- Explain lambda, alpha (hyperparameter)
- !! Explain the difference between L1 and L2 regularization

{TO-DO}: Improve the whole regularization section

Let's add some regularization to our previous model:

Let's create a logistic regression model in scikit-learn and inspect its parameters:

In [14]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
vars(clf)

{'penalty': 'l2',
 'dual': False,
 'tol': 0.0001,
 'C': 1.0,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'class_weight': None,
 'random_state': None,
 'solver': 'lbfgs',
 'max_iter': 100,
 'multi_class': 'auto',
 'verbose': 0,
 'warm_start': False,
 'n_jobs': None,
 'l1_ratio': None}

As you can see, by default scikit-learn penalizes logistic regression models using _Ridge (L2)_ regularization. If you read the documentation of `LogisticRegression` [here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) you will also notice that the parameter `C` determines the strenght of the regularization applied. By default this value is set to 1. Lower values will indicate stronger regularization.

{TO-DO}: explain this in more detail

In [19]:
clf = LogisticRegression(penalty="l2", C=0.01).fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

0.9333333333333333
0.84


In [138]:
clf = LogisticRegression(penalty="l2", C=100).fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

0.92
0.92


How to implement this in a linear regression model?

{TO-DO: expand this}

In [93]:
from sklearn.linear_model import Ridge

ridge = Ridge().fit(X_train, y_train)
print(ridge.score(X_train, y_train))
print(ridge.score(X_test, y_test))

0.6884727381337423
0.1874785187965038


(TODO: Explain better this part)

The value of regularization that gives the best performance in not known beforehand. Some people search the best performance value manually using the whole training and testing dataset. This could lead to overestimating the real performance of the model. Instead, good practice would be to tune this parameter the same as the weights of the model. This leads us to understanding the concept of __hyper-parameter tuning__.

# Hyper-parameter tuning

- What are hyperparameters
- Why we should tune our hyperparametes using cross validation and not search them manually.
- Concept of Nested Cross Validation
- What is, and how to perform, `GridSearchCV`

In [132]:
from sklearn.datasets import make_moons

X, y = make_moons(noise=0.352, random_state=1, n_samples=100)

In [133]:
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.svm import SVC

param_grid = [
    {'kernel': ['linear']},
    {'kernel': ['poly'], 'degree': [2, 3]},
    {'kernel': ['rbf']}
]

svc = SVC(random_state=0)

cv = RepeatedStratifiedKFold(
    n_splits=10, n_repeats=10, random_state=0
)

search = GridSearchCV(
    estimator=svc, param_grid=param_grid,
    scoring='roc_auc', cv=cv
)
search.fit(X, y)

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=10, random_state=0),
             estimator=SVC(random_state=0),
             param_grid=[{'kernel': ['linear']},
                         {'degree': [2, 3], 'kernel': ['poly']},
                         {'kernel': ['rbf']}],
             scoring='roc_auc')

In [135]:
import pandas as pd

results_df = pd.DataFrame(search.cv_results_)
results_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_degree,params,split0_test_score,split1_test_score,split2_test_score,...,split93_test_score,split94_test_score,split95_test_score,split96_test_score,split97_test_score,split98_test_score,split99_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000899,0.000336,0.001060,0.000373,linear,NaN,{'kernel': 'linear'},0.96,0.84,0.76,...,0.76,0.92,1.00,0.92,0.92,1.00,0.84,0.9300,0.077846,2
1,0.000904,0.000205,0.000979,0.000233,poly,2,"{'degree': 2, 'kernel': 'poly'}",0.76,0.64,0.56,...,0.52,0.48,0.68,0.68,0.76,0.84,0.52,0.6852,0.169106,4
2,0.000840,0.000173,0.000959,0.000214,poly,3,"{'degree': 3, 'kernel': 'poly'}",1.00,0.72,0.76,...,0.68,0.96,1.00,0.96,0.92,1.00,0.56,0.9044,0.098776,3
3,0.000996,0.000280,0.001243,0.000448,rbf,NaN,{'kernel': 'rbf'},0.92,0.72,0.76,...,0.72,0.92,1.00,0.96,1.00,1.00,0.84,0.9400,0.079297,1


In [136]:
results_df = results_df.sort_values(by=['rank_test_score'])
results_df = (
    results_df
    .set_index(results_df["params"].apply(
        lambda x: "_".join(str(val) for val in x.values()))
    )
    .rename_axis('kernel')
)
results_df[
    ['params', 'rank_test_score', 'mean_test_score', 'std_test_score']
]

,params,rank_test_score,mean_test_score,std_test_score
kernel,,,,
rbf,{'kernel': 'rbf'},1,0.9400,0.079297
linear,{'kernel': 'linear'},2,0.9300,0.077846
3_poly,"{'degree': 3, 'kernel': 'poly'}",3,0.9044,0.098776
2_poly,"{'degree': 2, 'kernel': 'poly'}",4,0.6852,0.169106


Learn how to perform statistical evaluation over these examples [here](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_stats.html)

### Exercise

Can you use `GridSearchCV` to search for the best value of `C` for a logistic regression model?

#### Answer

# Check your knowledge

{TO-DO}

Load the dataset pre-processed in notebook 1 {give name}, and:

1. 
2. 
3. 

Answer the following questions:

1. 
2.
3.


# Additional resources

